Extract necessary Libraries

In [1]:
import pandas as pd
from glob import glob


Extract features from file name as additional columns to the existing dataframe

In [9]:
# Read single CSV file
# --------------------------------------------------------------
single_file_acc = pd.read_csv(
    "../../data/raw/MetaMotion/A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Accelerometer_12.500Hz_1.4.4.csv")
single_file_gyr = pd.read_csv(
    "../../data/raw/MetaMotion/A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Gyroscope_25.000Hz_1.4.4.csv")

# --------------------------------------------------------------
# List all data in data/raw/MetaMotion
# --------------------------------------------------------------
files = glob("../../data/raw/MetaMotion/*.csv")
len(files)
# --------------------------------------------------------------
# Extract features from filename
# --------------------------------------------------------------
data_path = "../../data/raw/MetaMotion\\"
f = files[0]

participant = f.split('-')[0].replace(data_path, '')
label = f.split('-')[1]
category = f.split('-')[2].rstrip('123').rstrip('_MetaWear_2019')


In [6]:
# read the csv file of f
df = pd.read_csv(f)

# add the newly extracted features as columns
df['participant'] = participant
df['label'] = label
df['category'] = category

# print the first few rows
df.head()


,epoch (ms),time (01:00),elapsed (s),x-axis (g),y-axis (g),z-axis (g),participant,label,category
0,1547219408431,2019-01-11T16:10:08.431,0.00,0.010,0.964,-0.087,A,bench,heavy
1,1547219408511,2019-01-11T16:10:08.511,0.08,0.000,0.961,-0.069,A,bench,heavy
2,1547219408591,2019-01-11T16:10:08.591,0.16,0.001,0.974,-0.087,A,bench,heavy
3,1547219408671,2019-01-11T16:10:08.671,0.24,-0.012,0.971,-0.084,A,bench,heavy
4,1547219408751,2019-01-11T16:10:08.751,0.32,-0.013,0.954,-0.094,A,bench,heavy


We will be using what we have learnt above to create a function to extract these feature from all the files 

In [16]:
# create empty dataframes
acc_df = pd.DataFrame()
gyr_df = pd.DataFrame()

# set the counter for the number of set
acc_set = 1
gyr_set = 1
# apply the for loop
for f in files:
    participant = f.split('-')[0].replace(data_path, '')
    label = f.split('-')[1]
    category = f.split('-')[2].rstrip('123').rstrip('_MetaWear_2019')

    # read the csv file of f
    df = pd.read_csv(f)

    # add the newly extracted features as columns
    df['participant'] = participant
    df['label'] = label
    df['category'] = category

    # append to the acc_df and gyr_df
    if 'Accelerometer' in f:
        df['set'] = acc_set
        acc_set += 1
        acc_df = pd.concat([acc_df, df])
    if 'Gyroscope' in f:
        df['set'] = gyr_set
        gyr_set += 1
        gyr_df = pd.concat([gyr_df, df])

    # print(category)


In [17]:
acc_df


,epoch (ms),time (01:00),elapsed (s),x-axis (g),y-axis (g),z-axis (g),participant,label,category,set,Unnamed: 0
0,1.547219e+12,2019-01-11T16:10:08.431,0.00,0.010,0.964,-0.087,A,bench,heavy,1,NaN
1,1.547219e+12,2019-01-11T16:10:08.511,0.08,0.000,0.961,-0.069,A,bench,heavy,1,NaN
2,1.547219e+12,2019-01-11T16:10:08.591,0.16,0.001,0.974,-0.087,A,bench,heavy,1,NaN
3,1.547219e+12,2019-01-11T16:10:08.671,0.24,-0.012,0.971,-0.084,A,bench,heavy,1,NaN
4,1.547219e+12,2019-01-11T16:10:08.751,0.32,-0.013,0.954,-0.094,A,bench,heavy,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...
237,1.547666e+12,2019-01-16 20:14:23.089,18.96,0.012,0.596,0.815,E,squat,heavy,94,NaN
238,1.547666e+12,2019-01-16 20:14:23.169,19.04,0.009,0.528,0.821,E,squat,heavy,94,NaN
239,1.547666e+12,2019-01-16 20:14:23.249,19.12,0.015,0.554,0.746,E,squat,heavy,94,NaN
240,1.547666e+12,2019-01-16 20:14:23.329,19.20,0.006,0.574,0.824,E,squat,heavy,94,NaN
